In [1]:
import numpy as np
import tensorflow 
import keras
import matplotlib.pyplot as plt

In [2]:
def define_discriminator(in_shape = (28, 28, 1)):
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(30, (3, 3), strides = (1, 1), padding = 'valid', input_shape = in_shape))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Conv2D(30, (3, 3), strides = (1, 1), padding = 'valid'))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Conv2D(30, (3, 3), strides = (1, 1), padding = 'valid'))
    model.add(keras.layers.MaxPool2D((3, 3), strides = (1, 1), padding = 'valid'))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1, activation = 'sigmoid'))
    
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [3]:
def define_generator(noise):
    gen_model = keras.Sequential()
    gen_model.add(keras.layers.Dense(150, input_dim = noise))
    gen_model.add(keras.layers.LeakyReLU(alpha = 0.2))
    gen_model.add(keras.layers.Dropout(0.3))
    gen_model.add(keras.layers.Dense(250))
    gen_model.add(keras.layers.LeakyReLU(alpha = 0.2))
    gen_model.add(keras.layers.Dropout(0.3))
    gen_model.add(keras.layers.Dense(450))
    gen_model.add(keras.layers.LeakyReLU(alpha = 0.2))
    gen_model.add(keras.layers.Dropout(0.3))
    gen_model.add(keras.layers.Dense(600))
    gen_model.add(keras.layers.LeakyReLU(alpha = 0.2))
    gen_model.add(keras.layers.Dropout(0.3))
    gen_model.add(keras.layers.Dense(700))
    gen_model.add(keras.layers.LeakyReLU(alpha = 0.2))
    gen_model.add(keras.layers.Dropout(0.3))
    gen_model.add(keras.layers.Dense(784, activation = 'sigmoid'))
    gen_model.add(keras.layers.Reshape((28, 28, 1)))
#     gen_model.add(keras.activations.sigmoid())
    gen_model.add(keras.layers.Dropout(0.3))
    return gen_model

In [4]:
def define_gan(g_model, d_model):
    d_model.trainable = False
    model = keras.Sequential()
    model.add(g_model)
    model.add(d_model)
    
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [5]:
def load_real_samples():
    (train_X, _), (_, _) = keras.datasets.mnist.load_data()
    train_X = np.expand_dims(train_X, axis = -1)
    train_X = train_X/255
    return train_X

In [6]:
def generate_real_samples(dataset, n_samples):
    idx = np.random.randint(0, dataset.shape[0], n_samples)
    X = dataset[idx]
    Y = np.ones((n_samples, 1))
    return X, Y

In [7]:
def generate_latent_points(latent_dim, n_samples):
    noise = np.random.randn(latent_dim * n_samples)
    noise = noise.reshape(n_samples, latent_dim)
    return noise

In [8]:
def generate_fake_samples(g_model, latent_dim, n_samples):
    g_input = generate_latent_points(latent_dim, n_samples)
    fake_data = g_model.predict(g_input)
    Y = np.zeros((n_samples, 1))
    return fake_data, Y

In [9]:
def visualise(examples, epoch, n = 10):
    for i in range(n * n):
        plt.subplot(n, n, i + 1)
        plt.imshow(examples[i, :, :, :], cmap = 'gray')
    file_name = f'C:/Users/bhatt/OneDrive/Desktop/whisper-main/DS_PP_Classroom_14_Oct/Machine Learning/{epoch}_EPOCH.jpg'
    plt.savefig(file_name)

In [10]:
def evaluate(g_model, d_model, dataset, latent_dim, epoch, batch = 100):
    X_real, Y_real = generate_real_samples(dataset, batch)
    
    _, real_acc = d_model.evaluate(X_real, Y_real)
    
    X_fake, Y_fake = generate_fake_samples(g_model, latent_dim, batch)
    
    _, fake_acc = d_model.evaluate(X_fake, Y_fake)
    
    print(f'Accuracy real: {real_acc}, Accuracy fake: {fake_acc}')
    
    visualise(X_fake, epoch)

In [11]:
def train(g_model, d_model, gan_model, dataset, latent_dim, epoch, batch_size = 100):
    batch_per_epoch = int(dataset.shape[0]/batch_size)
    half_batch = int(batch_size/2)
    
    for i in range(epoch):
        for j in range(batch_per_epoch):
            X_real, Y_real = generate_real_samples(dataset, batch_size)
            
            X_fake, Y_fake = generate_fake_samples(g_model, latent_dim, batch_size)
            
            d_loss_real, _ = d_model.train_on_batch(X_real, Y_real)
            d_loss_fake, _ = d_model.train_on_batch(X_fake, Y_fake)
            
            total_d_loss = d_loss_fake + d_loss_real
            
            X_noise = generate_latent_points(latent_dim, batch_size)
            
            Y_g_model = np.ones((batch_size, 1))
            
            g_loss, _ = gan_model.train_on_batch(X_noise, Y_g_model)
            
            print(f'On: {i} D_loss: {total_d_loss}, G_loss: {g_loss}')
        
        if (i % 10 == 0):
            evaluate(g_model, d_model, dataset, latent_dim, i)

In [ ]:
latent_dim = 100

d_model = define_discriminator()

g_model = define_generator(latent_dim)

gan_model = define_gan(g_model, d_model)

dataset = load_real_samples()

train(g_model, d_model, gan_model, dataset, latent_dim, 100)

4/4 [==============================] - 0s 7ms/step
D_loss: 2.551741361618042, G_loss: 0.288585901260376
4/4 [==============================] - 0s 6ms/step
D_loss: 1.3660090267658234, G_loss: 0.625067949295044
4/4 [==============================] - 0s 3ms/step
D_loss: 1.1908301711082458, G_loss: 0.9417855739593506
4/4 [==============================] - 0s 5ms/step
D_loss: 1.165671020746231, G_loss: 1.1960331201553345
4/4 [==============================] - 0s 5ms/step
D_loss: 1.1847006976604462, G_loss: 1.2567377090454102
4/4 [==============================] - 0s 6ms/step
D_loss: 1.1529564261436462, G_loss: 1.2024476528167725
4/4 [==============================] - 0s 3ms/step
D_loss: 1.2020696997642517, G_loss: 0.9702001810073853
4/4 [==============================] - 0s 4ms/step
D_loss: 1.2775627374649048, G_loss: 0.7477110028266907
4/4 [==============================] - 0s 4ms/step
D_loss: 1.3869941234588623, G_loss: 0.6618783473968506
4/4 [==============================] - 0s 4ms/step

4/4 [==============================] - 0s 5ms/step
D_loss: 0.005762535147368908, G_loss: 6.179190158843994
4/4 [==============================] - 0s 6ms/step
D_loss: 0.008316247025504708, G_loss: 6.144450664520264
4/4 [==============================] - 0s 5ms/step
D_loss: 0.02040737634524703, G_loss: 6.331638336181641
4/4 [==============================] - 0s 4ms/step
D_loss: 0.006951572373509407, G_loss: 6.239859104156494
4/4 [==============================] - 0s 4ms/step
D_loss: 0.020977484062314034, G_loss: 5.939309120178223
4/4 [==============================] - 0s 5ms/step
D_loss: 0.007351472042500973, G_loss: 6.190205097198486
4/4 [==============================] - 0s 5ms/step
D_loss: 0.005413575941929594, G_loss: 6.008523464202881
4/4 [==============================] - 0s 4ms/step
D_loss: 0.007628569845110178, G_loss: 6.099696159362793
4/4 [==============================] - 0s 7ms/step
D_loss: 0.00950318155810237, G_loss: 5.977667331695557
4/4 [==============================] - 

4/4 [==============================] - 0s 3ms/step
D_loss: 0.01101842150092125, G_loss: 8.556905746459961
4/4 [==============================] - 0s 3ms/step
D_loss: 0.010642579291015863, G_loss: 8.771893501281738
4/4 [==============================] - 0s 3ms/step
D_loss: 0.012876095715910196, G_loss: 8.664440155029297
4/4 [==============================] - 0s 3ms/step
D_loss: 0.006490490399301052, G_loss: 8.827157974243164
4/4 [==============================] - 0s 3ms/step
D_loss: 0.01161647168919444, G_loss: 8.38576889038086
4/4 [==============================] - 0s 3ms/step
D_loss: 0.01047383900731802, G_loss: 8.503954887390137
4/4 [==============================] - 0s 3ms/step
D_loss: 0.009210084564983845, G_loss: 8.927774429321289
4/4 [==============================] - 0s 3ms/step
D_loss: 0.0066378258634358644, G_loss: 8.906476974487305
4/4 [==============================] - 0s 3ms/step
D_loss: 0.0066599673591554165, G_loss: 8.844019889831543
4/4 [==============================] - 

4/4 [==============================] - 0s 3ms/step
D_loss: 0.001376247382722795, G_loss: 11.440546035766602
4/4 [==============================] - 0s 3ms/step
D_loss: 0.0015207456890493631, G_loss: 11.585277557373047
4/4 [==============================] - 0s 3ms/step
D_loss: 0.0018798456294462085, G_loss: 11.913542747497559
4/4 [==============================] - 0s 4ms/step
D_loss: 0.0011065670987591147, G_loss: 11.789176940917969
4/4 [==============================] - 0s 3ms/step
D_loss: 0.0009108995727729052, G_loss: 11.757912635803223
4/4 [==============================] - 0s 3ms/step
D_loss: 0.0036213091807439923, G_loss: 11.858636856079102
4/4 [==============================] - 0s 3ms/step
D_loss: 0.0018269276479259133, G_loss: 12.019499778747559
4/4 [==============================] - 0s 4ms/step
D_loss: 0.003929427883122116, G_loss: 11.889077186584473
4/4 [==============================] - 0s 3ms/step
D_loss: 0.0008697478115209378, G_loss: 11.660261154174805
4/4 [================

D_loss: 0.0027184647624380887, G_loss: 12.583481788635254
4/4 [==============================] - 0s 3ms/step
D_loss: 0.001376157539198175, G_loss: 12.008520126342773
4/4 [==============================] - 0s 3ms/step
D_loss: 0.0010800287709571421, G_loss: 12.423503875732422
4/4 [==============================] - 0s 3ms/step
D_loss: 0.0018360656686127186, G_loss: 12.336030006408691
4/4 [==============================] - 0s 4ms/step
D_loss: 0.001502304570749402, G_loss: 12.078325271606445
4/4 [==============================] - 0s 3ms/step
D_loss: 0.014585714903660119, G_loss: 12.397404670715332
4/4 [==============================] - 0s 3ms/step
D_loss: 0.0020863335812464356, G_loss: 11.90900993347168
4/4 [==============================] - 0s 3ms/step
D_loss: 0.001403121161274612, G_loss: 12.196364402770996
4/4 [==============================] - 0s 3ms/step
D_loss: 0.002375989541178569, G_loss: 11.433147430419922
4/4 [==============================] - 0s 3ms/step
D_loss: 0.002782261697575

...

4/4 [==============================] - 0s 2ms/step
D_loss: 1.1293451308673486e-10, G_loss: 35.67433166503906
4/4 [==============================] - 0s 3ms/step
D_loss: 1.2751782160584646e-10, G_loss: 36.342044830322266
4/4 [==============================] - 0s 2ms/step
D_loss: 3.09278043496633e-10, G_loss: 35.489871978759766
4/4 [==============================] - 0s 3ms/step
D_loss: 1.3782121971450678e-10, G_loss: 35.70539855957031
4/4 [==============================] - 0s 2ms/step
D_loss: 1.2511250130527174e-10, G_loss: 35.181488037109375
4/4 [==============================] - 0s 2ms/step
D_loss: 7.73868231645434e-11, G_loss: 35.67827224731445
4/4 [==============================] - 0s 3ms/step
D_loss: 7.701425972915944e-11, G_loss: 35.61506271362305
4/4 [==============================] - 0s 3ms/step
D_loss: 9.306365416792903e-11, G_loss: 36.551231384277344
4/4 [==============================] - 0s 4ms/step
D_loss: 1.1545979450300337e-10, G_loss: 35.91557312011719
4/4 [================

4/4 [==============================] - 0s 2ms/step
D_loss: 2.343562113577176e-10, G_loss: 35.47636413574219
4/4 [==============================] - 0s 4ms/step
D_loss: 4.1541043521720056e-10, G_loss: 35.620365142822266
4/4 [==============================] - 0s 2ms/step
D_loss: 1.0622792204792279e-10, G_loss: 35.99126052856445
4/4 [==============================] - 0s 3ms/step
D_loss: 1.885293613525284e-10, G_loss: 35.071388244628906
4/4 [==============================] - 0s 2ms/step
D_loss: 2.3466965870522216e-10, G_loss: 36.374237060546875
4/4 [==============================] - 0s 3ms/step
D_loss: 3.6009596536155583e-10, G_loss: 36.861305236816406
4/4 [==============================] - 0s 2ms/step
D_loss: 5.11968946403929e-10, G_loss: 36.083866119384766
4/4 [==============================] - 0s 2ms/step
D_loss: 1.7222273889123951e-10, G_loss: 35.62527847290039
4/4 [==============================] - 0s 3ms/step
D_loss: 9.402962093576616e-11, G_loss: 35.384849548339844
4/4 [=============

4/4 [==============================] - 0s 3ms/step
D_loss: 0.00027502889863524583, G_loss: 12.438851356506348
4/4 [==============================] - 0s 3ms/step
D_loss: 0.0005994772072902661, G_loss: 13.068215370178223
4/4 [==============================] - 0s 3ms/step
D_loss: 0.0002784224634530581, G_loss: 12.286646842956543
4/4 [==============================] - 0s 3ms/step
D_loss: 0.0002201296738835863, G_loss: 12.84653091430664
4/4 [==============================] - 0s 3ms/step
D_loss: 0.0009338671807243625, G_loss: 13.108807563781738
4/4 [==============================] - 0s 2ms/step
D_loss: 0.00017727873637341426, G_loss: 12.8828763961792
4/4 [==============================] - 0s 2ms/step
D_loss: 0.0004816695582121748, G_loss: 13.090201377868652
4/4 [==============================] - 0s 3ms/step
D_loss: 0.0002798564383868339, G_loss: 13.294353485107422
4/4 [==============================] - 0s 3ms/step
D_loss: 0.0003640934883157114, G_loss: 12.653725624084473
4/4 [===============

4/4 [==============================] - 0s 2ms/step
D_loss: 0.00015355073265131363, G_loss: 14.148262977600098
4/4 [==============================] - 0s 3ms/step
D_loss: 2.542347648263234e-05, G_loss: 13.927201271057129
4/4 [==============================] - 0s 3ms/step
D_loss: 0.0001008286417389718, G_loss: 14.090463638305664
4/4 [==============================] - 0s 3ms/step
D_loss: 3.9811941796965124e-05, G_loss: 14.759407043457031
4/4 [==============================] - 0s 3ms/step
D_loss: 6.09819843039336e-05, G_loss: 14.671982765197754
4/4 [==============================] - 0s 2ms/step
D_loss: 8.063084534632093e-05, G_loss: 14.356114387512207
4/4 [==============================] - 0s 2ms/step
D_loss: 3.798954276438132e-05, G_loss: 13.519966125488281
4/4 [==============================] - 0s 3ms/step
D_loss: 9.045683766122184e-05, G_loss: 14.685637474060059
4/4 [==============================] - 0s 3ms/step
D_loss: 3.661261871458297e-05, G_loss: 14.213293075561523
4/4 [=============

4/4 [==============================] - 0s 3ms/step
D_loss: 1.9421700927112073e-05, G_loss: 15.682988166809082
4/4 [==============================] - 0s 2ms/step
D_loss: 2.9996777887024477e-05, G_loss: 15.09622573852539
4/4 [==============================] - 0s 3ms/step
D_loss: 1.6707863332576267e-05, G_loss: 15.490378379821777
4/4 [==============================] - 0s 3ms/step
D_loss: 1.7453614418632258e-05, G_loss: 15.628559112548828
4/4 [==============================] - 0s 3ms/step
D_loss: 1.8410784912557315e-05, G_loss: 14.773393630981445
4/4 [==============================] - 0s 2ms/step
D_loss: 8.162081212879125e-05, G_loss: 15.66879653930664
4/4 [==============================] - 0s 3ms/step
D_loss: 2.232105908026287e-05, G_loss: 15.005698204040527
4/4 [==============================] - 0s 2ms/step
D_loss: 4.417246239996279e-05, G_loss: 14.487500190734863
4/4 [==============================] - 0s 3ms/step
D_loss: 2.2688713215925294e-05, G_loss: 15.577988624572754
4/4 [==========